###  Capstone project - Final course

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## 1. Creating dataframe from wiki page

In [3]:
!conda install -c conda-forge beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [5]:
import bs4 as bs
import urllib.request
import pandas as pd

In [97]:
# Using beautifulsoup to read wiki HTML
wiki_source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
wiki_soup = bs.BeautifulSoup(wiki_source,'lxml')

pc_table = wiki_soup.find('table', attrs={'class':'wikitable sortable'})
pc_table_rows = pc_table.find_all('tr')

pc_data = []
for tr in pc_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) > 0:
        pc_data.append(row)


pc_df = pd.DataFrame(pc_data, columns=["PostalCode", "Borough", "Neighborhood"])

#Ignore cells with a borough that is Not assigned
pc_df = pc_df[pc_df.Borough != 'Not assigned']

#same postalCode will be combined into one row with the neighborhoods separated with a comm
pc_df = pc_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

#if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
pc_df.Neighborhood = pc_df.Neighborhood.replace('Not assigned', pc_df.Borough)

pc_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [98]:
#use the .shape method to print the number of rows of your dataframe
pc_df.shape

(103, 3)

## 2. Creating dataframe with Geocoder

In [109]:
# get data from csv
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.rename(columns={"Postal Code": "PostalCode"}, inplace = True)
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [149]:
#creating dataframe with Geocoder

neighborhood_df = pd.merge(pc_df, geo_df, on="PostalCode")

neighborhood_df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## 3. Explore Dataset

In [114]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.4.0-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 100% 
branca-0.4.0         | 26 KB     | ##################################### | 100% 
folium-0.10.1        | 59 KB   

In [240]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import requests
from pandas.io.json import json_normalize
import math

#### Use geopy library to get the latitude and longitude values of Toronto

In [151]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [158]:
neighborhood_df.dtypes

PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [157]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Borough'], neighborhood_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the __Scarborough__ data

In [166]:
scarborough_data = neighborhood_df[neighborhood_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [162]:
#Let's get the geographical coordinates of Scarborough

address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [163]:
# create map of Scarborough  using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### Define Foursquare Credentials and Version

In [167]:
CLIENT_ID = 'MFN5JY3ENCCAR40NV2PTO3QM30UQWPECJH244AR1UZVFN0TC'
CLIENT_SECRET = 'ENHQE5Z0VVC2SGD1JYCVFYZBDYFQIHXQGON040OC0TZKSFAS'
VERSION = '20180605'

#### Explore the first neighborhood in our dataframe

In [169]:
scarborough_data.loc[0, 'Neighborhood']

'Rouge,Malvern'

In [170]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


#### get the top 100 venues that are in Rouge,Malvern within a radius of 500 meters.

In [181]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # for more venues
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MFN5JY3ENCCAR40NV2PTO3QM30UQWPECJH244AR1UZVFN0TC&client_secret=ENHQE5Z0VVC2SGD1JYCVFYZBDYFQIHXQGON040OC0TZKSFAS&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

In [182]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6f37a992393500229d0b95'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '579a91b3498e9bd833afa78a',
       'name': "Wendy's",
       'location': {'address': '8129 Sheppard Avenue',
        'lat': 43.8020084,
        'lng': -79.1980797,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.8020084,
          'lng': -79.1980797}],
        'distance': 600,
        'postalCode': 'M1B 6A3',
        'cc': 'CA',
        '

In [183]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [184]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.802008,-79.198080
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
3,Harvey's,Restaurant,43.800020,-79.198307
4,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607


In [185]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


## 4. Explore Neighborhoods in Scarborough

In [186]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [188]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'], latitudes=scarborough_data['Latitude'],longitudes=scarborough_data['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [190]:
print(scarborough_venues.shape)
scarborough_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [191]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest,Cliffside,Scarborough Village West",3,3,3,3,3,3
"Dorset Park,Scarborough Town Centre,Wexford Heights",5,5,5,5,5,5
"East Birchmount Park,Ionview,Kennedy Park",6,6,6,6,6,6


In [192]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


## 5. Analyze Each Neighborhood

In [194]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [195]:
scarborough_onehot.shape

(86, 55)

In [196]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.0,0.000000,0.00,...,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
2,"Birch Cliff,Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.25,...,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
3,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.0,0.000000,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.125000,0.000000,0.0
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.2,0.100000,0.00,...,0.000,0.000000,0.00,0.00,0.1,0.000,0.000000,0.000000,0.000000,0.0
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000000,0.000,0.000,0.083333,0.0,0.000000,0.0,0.000000,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.083333,0.000000,0.0
6,"Cliffcrest,Cliffside,Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.2
8,"East Birchmount Park,Ionview,Kennedy Park",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.166667,0.00,...,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
9,"Guildwood,Morningside,West Hill",0.000000,0.000,0.000,0.125000,0.0,0.125000,0.0,0.000000,0.00,...,0.125,0.000000,0.00,0.00,0.0,0.125,0.000000,0.000000,0.000000,0.0


In [197]:
scarborough_grouped.shape

(16, 55)

In [199]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4    Korean Restaurant   0.0


----Birch Cliff,Cliffside West----
                   venue  freq
0                   Café  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.12
1     Hakka Restaurant  0.12
2               Bakery  0.12
3                 Bank  0.12
4          Gas Station  0.12


----Clairlea,Golden Mile,Oakridge----
            venue  freq
0          Bakery   0.2
1        Bus Line   0.2
2    

In [227]:
# create the new dataframe and display the top 10 venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,Gas Station
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store
4,"Clairlea,Golden Mile,Oakridge",Bus Line,Bakery,Park,Intersection,Metro Station,Bus Station,Ice Cream Shop,Soccer Field,Breakfast Spot,Electronics Store


## 6. Cluster Neighborhoods

In [228]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [229]:
scarborough_grouped_clustering.head()

,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,...,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,0.0,0.000,0.000,0.000,0.0,0.25,0.0,0.0,0.00,0.000,...,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,0.0,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.00,0.000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.0
2,0.0,0.000,0.000,0.000,0.0,0.00,0.0,0.0,0.25,0.000,...,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,0.0,0.125,0.125,0.125,0.0,0.00,0.0,0.0,0.00,0.125,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.125,0.0,0.0
4,0.0,0.000,0.200,0.000,0.0,0.00,0.2,0.1,0.00,0.000,...,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.000,0.0,0.0


In [230]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Intersection,Spa,Bank,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Vietnamese Restaurant,Department Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store


Finally, let's visualize the resulting clusters

In [257]:
#scarborough_merged['Cluster Labels'] = int(scarborough_merged['Cluster Labels'])
scarborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Intersection,Spa,Bank,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Vietnamese Restaurant,Department Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store


In [258]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
get_cluster = 0
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if math.isnan(cluster) == False: 
        get_cluster = int(cluster)-1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[get_cluster],
        fill=True,
        fill_color=rainbow[get_cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 7. Examine Clusters

#### Cluster 1

In [261]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,0.0,Grocery Store,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,Park,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [251]:
#### Cluster 2

In [262]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood,Morningside,West Hill",1.0,Intersection,Spa,Bank,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Vietnamese Restaurant,Department Store
3,Woburn,1.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
4,Cedarbrae,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store
6,"East Birchmount Park,Ionview,Kennedy Park",1.0,Department Store,Hobby Shop,Bus Station,Coffee Shop,Discount Store,Convenience Store,Vietnamese Restaurant,Hakka Restaurant,Gym,Grocery Store
7,"Clairlea,Golden Mile,Oakridge",1.0,Bus Line,Bakery,Park,Intersection,Metro Station,Bus Station,Ice Cream Shop,Soccer Field,Breakfast Spot,Electronics Store
9,"Birch Cliff,Cliffside West",1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,Gas Station
10,"Dorset Park,Scarborough Town Centre,Wexford He...",1.0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Spa,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station
11,"Maryvale,Wexford",1.0,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
12,Agincourt,1.0,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
13,"Clarks Corners,Sullivan,Tam O'Shanter",1.0,Pizza Place,Chinese Restaurant,Gas Station,Noodle House,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Thai Restaurant


#### Cluster 3

In [263]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek,Rouge Hill,Port Union",2.0,Bar,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


#### Cluster 4

In [264]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",3.0,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


#### Cluster 5

In [265]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffcrest,Cliffside,Scarborough Village West",4.0,American Restaurant,Motel,Movie Theater,Ice Cream Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
